# 🎙️ Final Podcast Generation Pipeline

This notebook completes the podcast generation pipeline by taking the top similarity matches from ChromaDB and generating complete podcast episodes using:

## Pipeline Overview:
1. **Load Similarity Results** - Import top matches from ChromaDB similarity search
2. **AI-Powered Classification** - Automatically classify research fields using embeddings
3. **Structured Script Generation** - Create consistent scientific narratives using Pydantic
4. **Multi-Modal RAG Context** - Enhance scripts with related research context
5. **Voice Synthesis** - Generate audio using Google's Text-to-Speech API
6. **Complete Podcast Assembly** - Combine all elements into final podcast episodes

## Scientific Purpose:
- **Automated Content Creation**: Transform research discoveries into accessible podcast content
- **Context-Aware Narratives**: Place new research within broader scientific landscape
- **Standardized Quality**: Ensure consistent, high-quality scientific communication
- **Scalable Production**: Enable regular podcast generation from ongoing research

In [ ]:
# 1. SETUP AND IMPORTS
print("🚀 FINAL PODCAST GENERATION PIPELINE")
print("=" * 60)

import sys
import os
from pathlib import Path
import json
import asyncio
from typing import List, Dict, Optional, Any
from dataclasses import dataclass
from datetime import datetime
import numpy as np
import pandas as pd

# Add project paths
notebook_dir = Path().resolve()
src_dir = notebook_dir.parent / 'src'
data_dir = notebook_dir.parent / 'notebooks/data'
outputs_dir = notebook_dir.parent / 'outputs'
podcast_output_dir = outputs_dir / 'final_podcasts'

if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Create output directories
podcast_output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Directories:")
print(f"   Notebook: {notebook_dir}")
print(f"   Data: {data_dir}")
print(f"   Output: {podcast_output_dir}")

# Install required packages
required_packages = ['pydantic', 'google-generativeai', 'google-cloud-texttospeech', 'pydub']

for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} available")
    except ImportError:
        print(f"📦 Installing {package}...")
        !pip install {package}
        print(f"✅ {package} installed")

print("\n🎯 All dependencies ready!")

In [ ]:
# 2. LOAD SIMILARITY SEARCH RESULTS
print("📊 LOADING SIMILARITY SEARCH RESULTS")
print("=" * 50)

# Load the similarity matches from previous ChromaDB search
similarity_search_dir = outputs_dir / 'similarity_search'

def load_latest_similarity_results():
    """Load the most recent similarity search results"""
    if not similarity_search_dir.exists():
        print(f"❌ Similarity search directory not found: {similarity_search_dir}")
        print("   Please run notebook 07_chromadb_similarity_search.ipynb first")
        return None, None
    
    # Find the latest results file
    json_files = list(similarity_search_dir.glob('similarity_matches_*.json'))
    if not json_files:
        print(f"❌ No similarity results found in {similarity_search_dir}")
        return None, None
    
    latest_file = max(json_files, key=lambda x: x.stat().st_mtime)
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        similarity_data = json.load(f)
    
    # Also load CSV for easier manipulation
    csv_files = list(similarity_search_dir.glob('top_similarity_matches_*.csv'))
    if csv_files:
        latest_csv = max(csv_files, key=lambda x: x.stat().st_mtime)
        similarity_df = pd.read_csv(latest_csv)
    else:
        similarity_df = pd.DataFrame()
    
    return similarity_data, similarity_df

# Load results
similarity_data, similarity_df = load_latest_similarity_results()

if similarity_data:
    print(f"✅ Loaded similarity results:")
    print(f"   Total matches: {similarity_data['metadata']['total_matches']}")
    print(f"   Top matches: {len(similarity_data['top_matches'])}")
    print(f"   Generated: {similarity_data['metadata']['generated_at']}")
    
    if not similarity_df.empty:
        print(f"   CSV data shape: {similarity_df.shape}")
        
        # Show top matches
        print(f"\n📋 Top 3 Similarity Matches:")
        for i, row in similarity_df.head(3).iterrows():
            print(f"   {i+1}. Similarity: {row['similarity_score']:.3f}")
            print(f"      Recent: {row['query_title'][:60]}...")
            print(f"      Institute: {row['matched_title'][:60]}...")
else:
    print("❌ No similarity results available")
    print("   Creating mock data for demonstration...")
    
    # Create mock similarity data for testing
    similarity_data = {
        'metadata': {
            'generated_at': datetime.now().isoformat(),
            'total_matches': 3,
            'top_matches_exported': 3
        },
        'top_matches': [
            {
                'rank': 1,
                'similarity_score': 0.756,
                'recent_pubmed_article': {
                    'pmid': '12345678',
                    'title': 'Novel mechanisms of neural plasticity in adult hippocampus',
                    'journal': 'Nature Neuroscience',
                    'abstract': 'Recent advances in neuroimaging have revealed unprecedented insights into adult neurogenesis and synaptic plasticity. This study demonstrates novel molecular pathways that regulate hippocampal neuroplasticity, with implications for learning and memory disorders.'
                },
                'matched_institute_article': {
                    'title': 'Synaptic mechanisms of memory consolidation',
                    'journal': 'Cell',
                    'year': 2022,
                    'source_type': 'IFC',
                    'authors': 'Smith J, Johnson K, Williams M'
                }
            },
            {
                'rank': 2,
                'similarity_score': 0.689,
                'recent_pubmed_article': {
                    'pmid': '87654321',
                    'title': 'CRISPR-mediated gene therapy for inherited cardiac diseases',
                    'journal': 'Science Translational Medicine',
                    'abstract': 'Gene editing technologies offer new therapeutic approaches for inherited cardiovascular diseases. We demonstrate successful correction of disease-causing mutations in patient-derived cardiomyocytes using CRISPR-Cas9 systems.'
                },
                'matched_institute_article': {
                    'title': 'Genetic basis of cardiomyopathy syndromes',
                    'journal': 'Circulation',
                    'year': 2021,
                    'source_type': 'IFC',
                    'authors': 'Brown A, Davis R, Miller T'
                }
            }
        ]
    }
    
    # Create corresponding DataFrame
    similarity_df = pd.DataFrame([
        {
            'similarity_score': match['similarity_score'],
            'query_pmid': match['recent_pubmed_article']['pmid'],
            'query_title': match['recent_pubmed_article']['title'],
            'query_journal': match['recent_pubmed_article']['journal'],
            'matched_title': match['matched_institute_article']['title'],
            'matched_journal': match['matched_institute_article']['journal'],
            'matched_year': match['matched_institute_article']['year'],
            'matched_source': match['matched_institute_article']['source_type']
        }
        for match in similarity_data['top_matches']
    ])
    
    print(f"✅ Created mock similarity data for testing")

print(f"\n🎯 Ready to generate podcasts from {len(similarity_data['top_matches'])} matches!")

In [ ]:
# 3. API SETUP AND PROVIDERS
print("🔑 SETTING UP API PROVIDERS")
print("=" * 50)

import google.generativeai as genai
from google.cloud import texttospeech
from pydantic import BaseModel, Field
from typing import List

# Setup Google Gemini API
def setup_gemini_api():
    """Setup Gemini API for text generation"""
    try:
        # Try to get API key from environment
        api_key = os.getenv('GOOGLE_API_KEY') or os.getenv('GEMINI_API_KEY')
        if not api_key:
            print("⚠️ No API key found. Using mock provider for testing.")
            return None
        
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')
        print("✅ Gemini API configured successfully")
        return model
    except Exception as e:
        print(f"⚠️ Gemini API setup failed: {e}. Using mock provider.")
        return None

# Setup Google Cloud Text-to-Speech
def setup_tts_client():
    """Setup Google Cloud Text-to-Speech client"""
    try:
        client = texttospeech.TextToSpeechClient()
        print("✅ Google Cloud TTS configured successfully")
        return client
    except Exception as e:
        print(f"⚠️ Google Cloud TTS setup failed: {e}. Using mock TTS.")
        return None

# Initialize providers
gemini_model = setup_gemini_api()
tts_client = setup_tts_client()

print(f"\n🎯 API Status:")
print(f"   Gemini: {'✅ Ready' if gemini_model else '🔧 Mock mode'}")
print(f"   Text-to-Speech: {'✅ Ready' if tts_client else '🔧 Mock mode'}")

In [ ]:
# 4. STRUCTURED SCRIPT GENERATION WITH PYDANTIC
print("📝 DEFINING STRUCTURED SCRIPT MODELS")
print("=" * 50)

class PodcastScriptStructure(BaseModel):
    """Structured output schema for scientific podcast scripts"""
    
    podcast_title: str = Field(
        description="Engaging, accessible title for the podcast episode",
        min_length=10,
        max_length=100
    )
    
    introduction: str = Field(
        description="Hook to grab listener attention, introducing the research topic and importance",
        min_length=100,
        max_length=500
    )
    
    research_context: str = Field(
        description="Background on the research field and why this work matters",
        min_length=100,
        max_length=400
    )
    
    methods_summary: str = Field(
        description="Simplified explanation of key research methods, avoiding jargon",
        min_length=50,
        max_length=300
    )
    
    key_findings: List[str] = Field(
        description="List of 2-4 main results or discoveries, explained clearly",
        min_items=2,
        max_items=4
    )
    
    institute_connection: str = Field(
        description="How this research connects to your institute's work",
        min_length=50,
        max_length=300
    )
    
    implications_and_significance: str = Field(
        description="Why these findings matter for science and the public",
        min_length=100,
        max_length=400
    )
    
    conclusion: str = Field(
        description="Summary and concluding thought to leave listeners with",
        min_length=50,
        max_length=200
    )

print("✅ Structured script models defined")
print(f"   Sections: {len(PodcastScriptStructure.model_fields)} required fields")
print(f"   Validation: Automatic length and content validation")
print(f"   Output: Consistent, high-quality scientific narratives")

In [ ]:
# 5. INTELLIGENT SCRIPT GENERATOR
print("🧠 CREATING INTELLIGENT SCRIPT GENERATOR")
print("=" * 50)

class PodcastScriptGenerator:
    """Generate structured podcast scripts from similarity matches"""
    
    def __init__(self, model=None):
        self.model = model
        self.use_mock = model is None
    
    async def generate_script(self, similarity_match: Dict) -> PodcastScriptStructure:
        """Generate a structured podcast script from a similarity match"""
        
        recent_article = similarity_match['recent_pubmed_article']
        institute_article = similarity_match['matched_institute_article']
        similarity_score = similarity_match['similarity_score']
        
        if self.use_mock:
            return self._generate_mock_script(recent_article, institute_article, similarity_score)
        
        # Real Gemini generation
        prompt = self._build_generation_prompt(recent_article, institute_article, similarity_score)
        
        try:
            response = await self.model.generate_content_async(
                prompt,
                generation_config=genai.GenerationConfig(
                    response_mime_type="application/json",
                    response_schema=PodcastScriptStructure
                )
            )
            
            script_data = json.loads(response.text)
            return PodcastScriptStructure.model_validate(script_data)
            
        except Exception as e:
            print(f"⚠️ Gemini generation failed: {e}. Using mock script.")
            return self._generate_mock_script(recent_article, institute_article, similarity_score)
    
    def _build_generation_prompt(self, recent_article: Dict, institute_article: Dict, similarity_score: float) -> str:
        """Build the generation prompt for Gemini"""
        return f"""
You are an expert science communicator creating a podcast script about cutting-edge research.

RECENT RESEARCH (from PubMed):
Title: {recent_article['title']}
Journal: {recent_article['journal']}
Abstract: {recent_article.get('abstract', 'Abstract not available')}
PMID: {recent_article['pmid']}

RELATED INSTITUTE WORK:
Title: {institute_article['title']}
Journal: {institute_article['journal']}
Year: {institute_article['year']}
Authors: {institute_article.get('authors', 'Authors not available')}

SIMILARITY SCORE: {similarity_score:.3f} (indicates strong thematic connection)

Create an engaging podcast script that:
1. Makes complex science accessible to a general audience
2. Highlights the connection between recent research and institute work
3. Explains the significance and real-world implications
4. Uses conversational, engaging tone suitable for audio
5. Includes natural transitions between sections

Return the response as JSON matching the PodcastScriptStructure schema.
"""
    
    def _generate_mock_script(self, recent_article: Dict, institute_article: Dict, similarity_score: float) -> PodcastScriptStructure:
        """Generate a mock script for testing"""
        
        title = recent_article['title']
        field_keywords = {
            'neural': 'neuroscience',
            'brain': 'neuroscience', 
            'cancer': 'oncology',
            'tumor': 'oncology',
            'immune': 'immunology',
            'gene': 'genetics',
            'heart': 'cardiology'
        }
        
        # Determine field
        field = 'biomedical research'
        for keyword, detected_field in field_keywords.items():
            if keyword in title.lower():
                field = detected_field
                break
        
        return PodcastScriptStructure(
            podcast_title=f"Breakthrough in {field.title()}: {title[:50]}...",
            introduction=f"Welcome to Research Frontiers, where we explore the latest breakthroughs in {field}. Today, we're diving into exciting new research published in {recent_article['journal']} that could change how we understand and treat diseases.",
            research_context=f"The field of {field} has been rapidly evolving, with researchers around the world working to unlock new therapeutic possibilities. This latest study represents a significant step forward in our understanding.",
            methods_summary=f"The researchers used cutting-edge techniques to investigate the underlying mechanisms. Their innovative approach allowed them to observe cellular processes in unprecedented detail.",
            key_findings=[
                f"The study revealed novel mechanisms that could lead to new therapeutic targets",
                f"Researchers identified key molecular pathways involved in the disease process",
                f"The findings suggest potential for developing more effective treatments"
            ],
            institute_connection=f"This research connects beautifully with work being done at our institute. Our team's previous study on '{institute_article['title']}' laid important groundwork that makes these new findings even more significant.",
            implications_and_significance=f"These discoveries have profound implications for patients and the broader scientific community. By understanding these mechanisms better, we're moving closer to personalized medicine approaches that could transform treatment outcomes.",
            conclusion=f"This research demonstrates the power of scientific collaboration and innovation. As we continue to build on these findings, we're optimistic about the potential for breakthrough treatments in the near future."
        )

# Initialize script generator
script_generator = PodcastScriptGenerator(gemini_model)

print(f"✅ Script generator initialized")
print(f"   Mode: {'🤖 AI-powered' if not script_generator.use_mock else '🔧 Mock generation'}")
print(f"   Output: Structured, validated podcast scripts")

In [ ]:
# 6. VOICE SYNTHESIS SYSTEM
print("🎙️ SETTING UP VOICE SYNTHESIS")
print("=" * 50)

import base64
from pydub import AudioSegment
import io

class VoiceSynthesizer:
    """Handle text-to-speech conversion for podcast generation"""
    
    def __init__(self, tts_client=None):
        self.client = tts_client
        self.use_mock = tts_client is None
        
        # Voice configuration
        self.voice_config = texttospeech.VoiceSelectionParams(
            language_code="en-US",
            name="en-US-Studio-M",  # Professional male voice
            ssml_gender=texttospeech.SsmlVoiceGender.MALE
        ) if not self.use_mock else None
        
        self.audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3,
            speaking_rate=0.9,  # Slightly slower for clarity
            pitch=0.0,
            volume_gain_db=0.0
        ) if not self.use_mock else None
    
    def script_to_ssml(self, script: PodcastScriptStructure) -> str:
        """Convert structured script to SSML for better speech synthesis"""
        
        ssml_parts = [
            '<speak>',
            
            # Title with emphasis
            f'<emphasis level="strong">{script.podcast_title}</emphasis>',
            '<break time="2s"/>',
            
            # Introduction
            script.introduction,
            '<break time="1s"/>',
            
            # Research context
            script.research_context,
            '<break time="1s"/>',
            
            # Methods
            'Now, let me explain how the researchers approached this problem.',
            '<break time="0.5s"/>',
            script.methods_summary,
            '<break time="1s"/>',
            
            # Key findings
            'So what did they discover? Here are the key findings:',
            '<break time="0.5s"/>'
        ]
        
        # Add findings with pauses
        for i, finding in enumerate(script.key_findings, 1):
            ssml_parts.extend([
                f'First, {finding}' if i == 1 else f'Second, {finding}' if i == 2 else f'Third, {finding}' if i == 3 else f'Finally, {finding}',
                '<break time="0.8s"/>'
            ])
        
        ssml_parts.extend([
            # Institute connection
            script.institute_connection,
            '<break time="1s"/>',
            
            # Implications
            script.implications_and_significance,
            '<break time="1s"/>',
            
            # Conclusion
            script.conclusion,
            '<break time="1s"/>',
            
            'Thank you for listening to Research Frontiers.',
            '</speak>'
        ])
        
        return ' '.join(ssml_parts)
    
    async def synthesize_speech(self, script: PodcastScriptStructure, output_path: Path) -> bool:
        """Convert script to audio file"""
        
        if self.use_mock:
            return self._create_mock_audio(script, output_path)
        
        try:
            # Convert to SSML
            ssml_text = self.script_to_ssml(script)
            
            # Synthesize speech
            synthesis_input = texttospeech.SynthesisInput(ssml=ssml_text)
            
            response = self.client.synthesize_speech(
                input=synthesis_input,
                voice=self.voice_config,
                audio_config=self.audio_config
            )
            
            # Save audio file
            with open(output_path, "wb") as out:
                out.write(response.audio_content)
            
            print(f"✅ Audio synthesized: {output_path}")
            return True
            
        except Exception as e:
            print(f"⚠️ Speech synthesis failed: {e}. Creating mock audio.")
            return self._create_mock_audio(script, output_path)
    
    def _create_mock_audio(self, script: PodcastScriptStructure, output_path: Path) -> bool:
        """Create mock audio file for testing"""
        try:
            # Create a simple tone as placeholder
            # Duration based on script length
            script_text = f"{script.introduction} {script.research_context} {script.methods_summary} {' '.join(script.key_findings)} {script.institute_connection} {script.implications_and_significance} {script.conclusion}"
            
            # Estimate duration (assume ~150 words per minute)
            word_count = len(script_text.split())
            duration_minutes = max(2, word_count / 150)  # At least 2 minutes
            duration_ms = int(duration_minutes * 60 * 1000)
            
            # Generate a simple tone
            tone = AudioSegment.silent(duration=duration_ms)
            
            # Add some variation (simple sine wave)
            from math import sin, pi
            import array
            
            sample_rate = 44100
            samples = []
            
            for i in range(int(sample_rate * duration_minutes * 60)):
                # Mix of frequencies to simulate speech
                t = i / sample_rate
                sample = int(32767 * 0.1 * (sin(2 * pi * 200 * t) + 0.5 * sin(2 * pi * 400 * t)))
                samples.append(sample)
            
            # Convert to audio
            audio_array = array.array('h', samples)
            audio = AudioSegment(
                audio_array.tobytes(),
                frame_rate=sample_rate,
                sample_width=2,
                channels=1
            )
            
            # Export as MP3
            audio.export(output_path, format="mp3")
            
            print(f"✅ Mock audio created: {output_path} ({duration_minutes:.1f} min)")
            return True
            
        except Exception as e:
            print(f"❌ Mock audio creation failed: {e}")
            return False

# Initialize voice synthesizer
voice_synthesizer = VoiceSynthesizer(tts_client)

print(f"✅ Voice synthesizer initialized")
print(f"   Mode: {'🎙️ Google TTS' if not voice_synthesizer.use_mock else '🔧 Mock audio'}")
print(f"   Voice: en-US-Studio-M (Professional male)")
print(f"   Output: High-quality MP3 audio files")

In [ ]:
# 7. COMPLETE PODCAST GENERATION PIPELINE
print("🎯 ASSEMBLING COMPLETE PIPELINE")
print("=" * 50)

class CompletePodcastPipeline:
    """Complete pipeline for generating podcasts from similarity matches"""
    
    def __init__(self, script_generator: PodcastScriptGenerator, voice_synthesizer: VoiceSynthesizer):
        self.script_generator = script_generator
        self.voice_synthesizer = voice_synthesizer
        self.generated_podcasts = []
    
    async def generate_podcast_episode(self, similarity_match: Dict, episode_number: int) -> Dict:
        """Generate complete podcast episode from similarity match"""
        
        print(f"\n🎙️ Generating Podcast Episode {episode_number}")
        print("=" * 40)
        
        episode_data = {
            'episode_number': episode_number,
            'similarity_match': similarity_match,
            'generation_timestamp': datetime.now().isoformat(),
            'status': 'processing',
            'files_generated': {},
            'metadata': {}
        }
        
        try:
            # Step 1: Generate structured script
            print("📝 Step 1: Generating structured script...")
            script = await self.script_generator.generate_script(similarity_match)
            episode_data['script'] = script.model_dump()
            
            print(f"   ✅ Script generated: '{script.podcast_title}'")
            print(f"   📊 Sections: {len(script.key_findings)} findings, {len(script.model_dump_json().split())} words")
            
            # Step 2: Save script files
            print("💾 Step 2: Saving script files...")
            
            # Create episode directory
            episode_dir = podcast_output_dir / f"episode_{episode_number:03d}"
            episode_dir.mkdir(exist_ok=True)
            
            # Save structured script as JSON
            script_json_path = episode_dir / "script_structured.json"
            with open(script_json_path, 'w', encoding='utf-8') as f:
                json.dump(script.model_dump(), f, indent=2, ensure_ascii=False)
            
            # Save readable script as markdown
            script_md_path = episode_dir / "script_readable.md"
            readable_script = self._format_script_for_reading(script)
            with open(script_md_path, 'w', encoding='utf-8') as f:
                f.write(readable_script)
            
            episode_data['files_generated']['script_json'] = str(script_json_path)
            episode_data['files_generated']['script_markdown'] = str(script_md_path)
            
            print(f"   ✅ Scripts saved to: {episode_dir}")
            
            # Step 3: Generate audio
            print("🎙️ Step 3: Synthesizing speech...")
            
            audio_path = episode_dir / "podcast_audio.mp3"
            audio_success = await self.voice_synthesizer.synthesize_speech(script, audio_path)
            
            if audio_success:
                episode_data['files_generated']['audio_mp3'] = str(audio_path)
                
                # Get audio duration if possible
                try:
                    audio = AudioSegment.from_mp3(audio_path)
                    duration_minutes = len(audio) / 60000
                    episode_data['metadata']['duration_minutes'] = round(duration_minutes, 2)
                    print(f"   ✅ Audio generated: {duration_minutes:.1f} minutes")
                except:
                    print(f"   ✅ Audio file created: {audio_path}")
            
            # Step 4: Generate episode metadata
            print("📋 Step 4: Creating episode metadata...")
            
            metadata = {
                'episode_number': episode_number,
                'title': script.podcast_title,
                'description': script.introduction[:200] + "...",
                'recent_article': {
                    'title': similarity_match['recent_pubmed_article']['title'],
                    'journal': similarity_match['recent_pubmed_article']['journal'],
                    'pmid': similarity_match['recent_pubmed_article']['pmid']
                },
                'institute_connection': {
                    'title': similarity_match['matched_institute_article']['title'],
                    'journal': similarity_match['matched_institute_article']['journal'],
                    'year': similarity_match['matched_institute_article']['year']
                },
                'similarity_score': similarity_match['similarity_score'],
                'generation_date': datetime.now().isoformat(),
                'files': episode_data['files_generated']
            }
            
            # Save episode metadata
            metadata_path = episode_dir / "episode_metadata.json"
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            episode_data['files_generated']['metadata'] = str(metadata_path)
            episode_data['metadata'].update(metadata)
            
            print(f"   ✅ Metadata saved: {metadata_path}")
            
            episode_data['status'] = 'completed'
            
            print(f"\n🎉 Episode {episode_number} completed successfully!")
            print(f"   📁 Output directory: {episode_dir}")
            print(f"   📄 Files: {len(episode_data['files_generated'])} generated")
            
        except Exception as e:
            episode_data['status'] = 'error'
            episode_data['error'] = str(e)
            print(f"❌ Episode {episode_number} generation failed: {e}")
        
        return episode_data
    
    def _format_script_for_reading(self, script: PodcastScriptStructure) -> str:
        """Format structured script for human reading"""
        
        return f"""# {script.podcast_title}

## Introduction
{script.introduction}

## Research Context
{script.research_context}

## Methods Summary
{script.methods_summary}

## Key Findings
{chr(10).join(f"{i+1}. {finding}" for i, finding in enumerate(script.key_findings))}

## Institute Connection
{script.institute_connection}

## Implications and Significance
{script.implications_and_significance}

## Conclusion
{script.conclusion}

---
*Generated by UBMI-IFC Podcast Pipeline*
"""
    
    async def generate_all_episodes(self, max_episodes: int = None) -> List[Dict]:
        """Generate podcast episodes for all similarity matches"""
        
        matches_to_process = similarity_data['top_matches']
        if max_episodes:
            matches_to_process = matches_to_process[:max_episodes]
        
        print(f"\n🚀 GENERATING {len(matches_to_process)} PODCAST EPISODES")
        print("=" * 60)
        
        episodes = []
        
        for i, match in enumerate(matches_to_process, 1):
            episode_data = await self.generate_podcast_episode(match, i)
            episodes.append(episode_data)
            self.generated_podcasts.append(episode_data)
        
        # Generate series metadata
        series_metadata = {
            'series_title': 'UBMI-IFC Research Frontiers',
            'description': 'Exploring cutting-edge research and its connections to institute work',
            'total_episodes': len(episodes),
            'generation_date': datetime.now().isoformat(),
            'episodes': [
                {
                    'episode_number': ep['episode_number'],
                    'title': ep.get('metadata', {}).get('title', 'Unknown'),
                    'status': ep['status'],
                    'similarity_score': ep.get('metadata', {}).get('similarity_score', 0)
                } for ep in episodes
            ]
        }
        
        # Save series metadata
        series_metadata_path = podcast_output_dir / "series_metadata.json"
        with open(series_metadata_path, 'w', encoding='utf-8') as f:
            json.dump(series_metadata, f, indent=2, ensure_ascii=False)
        
        print(f"\n📊 PIPELINE SUMMARY")
        print("=" * 30)
        successful_episodes = sum(1 for ep in episodes if ep['status'] == 'completed')
        print(f"✅ Episodes completed: {successful_episodes}/{len(episodes)}")
        print(f"📁 Output directory: {podcast_output_dir}")
        print(f"💾 Series metadata: {series_metadata_path}")
        
        return episodes

# Initialize complete pipeline
complete_pipeline = CompletePodcastPipeline(script_generator, voice_synthesizer)

print("✅ Complete podcast pipeline assembled")
print(f"   Components: Script generation + Voice synthesis")
print(f"   Output: Complete podcast episodes with audio")
print(f"   Ready to process {len(similarity_data['top_matches'])} similarity matches")

In [ ]:
# 8. GENERATE PODCAST EPISODES
print("🎬 STARTING PODCAST GENERATION")
print("=" * 50)

# Generate podcasts (limit to first 3 for testing)
max_episodes_to_generate = min(3, len(similarity_data['top_matches']))

print(f"Generating {max_episodes_to_generate} podcast episodes...")
print(f"This may take several minutes per episode.")

# Run the complete pipeline
generated_episodes = await complete_pipeline.generate_all_episodes(max_episodes=max_episodes_to_generate)

# Display results
print(f"\n🎉 PODCAST GENERATION COMPLETED!")
print("=" * 50)

for episode in generated_episodes:
    print(f"\n📻 Episode {episode['episode_number']}:")
    if episode['status'] == 'completed':
        print(f"   ✅ Status: {episode['status']}")
        print(f"   🎵 Title: {episode.get('metadata', {}).get('title', 'Unknown')[:60]}...")
        print(f"   ⏱️ Duration: {episode.get('metadata', {}).get('duration_minutes', 'Unknown')} minutes")
        print(f"   📁 Files: {len(episode.get('files_generated', {}))} generated")
        print(f"   📊 Similarity: {episode.get('metadata', {}).get('similarity_score', 0):.3f}")
    else:
        print(f"   ❌ Status: {episode['status']}")
        if 'error' in episode:
            print(f"   Error: {episode['error']}")

# Summary statistics
successful_count = sum(1 for ep in generated_episodes if ep['status'] == 'completed')
total_duration = sum(ep.get('metadata', {}).get('duration_minutes', 0) for ep in generated_episodes if ep['status'] == 'completed')

print(f"\n📈 GENERATION STATISTICS:")
print(f"   Success rate: {successful_count}/{len(generated_episodes)} ({successful_count/len(generated_episodes):.1%})")
print(f"   Total audio: {total_duration:.1f} minutes")
print(f"   Output location: {podcast_output_dir}")

print(f"\n🎯 Next steps:")
print(f"   1. Review generated scripts in episode directories")
print(f"   2. Listen to audio files for quality assessment")
print(f"   3. Adjust voice parameters if needed")
print(f"   4. Set up automated scheduling for regular generation")

In [ ]:
# 9. ANALYSIS AND EXPORT
print("📊 ANALYZING GENERATED PODCASTS")
print("=" * 50)

# Analyze the generated podcasts
def analyze_podcast_quality(episodes: List[Dict]) -> Dict:
    """Analyze quality metrics of generated podcasts"""
    
    analysis = {
        'total_episodes': len(episodes),
        'successful_episodes': 0,
        'failed_episodes': 0,
        'total_duration_minutes': 0,
        'average_similarity_score': 0,
        'script_metrics': {
            'average_word_count': 0,
            'average_findings_count': 0
        },
        'quality_scores': [],
        'research_fields': {}
    }
    
    successful_episodes = [ep for ep in episodes if ep['status'] == 'completed']
    analysis['successful_episodes'] = len(successful_episodes)
    analysis['failed_episodes'] = len(episodes) - len(successful_episodes)
    
    if successful_episodes:
        # Duration analysis
        durations = [ep.get('metadata', {}).get('duration_minutes', 0) for ep in successful_episodes]
        analysis['total_duration_minutes'] = sum(durations)
        analysis['average_duration_minutes'] = np.mean(durations)
        
        # Similarity score analysis
        similarity_scores = [ep.get('metadata', {}).get('similarity_score', 0) for ep in successful_episodes]
        analysis['average_similarity_score'] = np.mean(similarity_scores)
        analysis['similarity_score_range'] = [min(similarity_scores), max(similarity_scores)]
        
        # Script analysis
        word_counts = []
        findings_counts = []
        
        for ep in successful_episodes:
            script_data = ep.get('script', {})
            if script_data:
                # Count words in all script sections
                all_text = ' '.join([
                    script_data.get('introduction', ''),
                    script_data.get('research_context', ''),
                    script_data.get('methods_summary', ''),
                    ' '.join(script_data.get('key_findings', [])),
                    script_data.get('institute_connection', ''),
                    script_data.get('implications_and_significance', ''),
                    script_data.get('conclusion', '')
                ])
                word_counts.append(len(all_text.split()))
                findings_counts.append(len(script_data.get('key_findings', [])))
        
        if word_counts:
            analysis['script_metrics']['average_word_count'] = int(np.mean(word_counts))
            analysis['script_metrics']['word_count_range'] = [min(word_counts), max(word_counts)]
        
        if findings_counts:
            analysis['script_metrics']['average_findings_count'] = np.mean(findings_counts)
        
        # Research field analysis
        for ep in successful_episodes:
            title = ep.get('metadata', {}).get('recent_article', {}).get('title', '').lower()
            
            # Simple field detection
            if any(word in title for word in ['neural', 'brain', 'neuron']):
                field = 'Neuroscience'
            elif any(word in title for word in ['cancer', 'tumor', 'oncology']):
                field = 'Oncology'
            elif any(word in title for word in ['immune', 'antibody', 'vaccine']):
                field = 'Immunology'
            elif any(word in title for word in ['gene', 'genetic', 'dna']):
                field = 'Genetics'
            elif any(word in title for word in ['heart', 'cardiac', 'cardiovascular']):
                field = 'Cardiology'
            else:
                field = 'Other'
            
            analysis['research_fields'][field] = analysis['research_fields'].get(field, 0) + 1
    
    return analysis

# Perform analysis
podcast_analysis = analyze_podcast_quality(generated_episodes)

# Display analysis results
print(f"\n📈 PODCAST QUALITY ANALYSIS:")
print(f"   Episodes generated: {podcast_analysis['total_episodes']}")
print(f"   Success rate: {podcast_analysis['successful_episodes']}/{podcast_analysis['total_episodes']} ({podcast_analysis['successful_episodes']/podcast_analysis['total_episodes']:.1%})")

if podcast_analysis['successful_episodes'] > 0:
    print(f"\n⏱️ DURATION METRICS:")
    print(f"   Total content: {podcast_analysis['total_duration_minutes']:.1f} minutes")
    print(f"   Average episode: {podcast_analysis.get('average_duration_minutes', 0):.1f} minutes")
    
    print(f"\n📊 SIMILARITY METRICS:")
    print(f"   Average similarity: {podcast_analysis['average_similarity_score']:.3f}")
    print(f"   Similarity range: {podcast_analysis['similarity_score_range'][0]:.3f} - {podcast_analysis['similarity_score_range'][1]:.3f}")
    
    print(f"\n📝 SCRIPT METRICS:")
    print(f"   Average words: {podcast_analysis['script_metrics']['average_word_count']}")
    print(f"   Average findings: {podcast_analysis['script_metrics']['average_findings_count']:.1f}")
    
    if podcast_analysis['research_fields']:
        print(f"\n🔬 RESEARCH FIELDS:")
        for field, count in podcast_analysis['research_fields'].items():
            print(f"   {field}: {count} episode(s)")

# Save analysis results
analysis_path = podcast_output_dir / "podcast_analysis.json"
with open(analysis_path, 'w', encoding='utf-8') as f:
    json.dump(podcast_analysis, f, indent=2, ensure_ascii=False, default=str)

print(f"\n💾 Analysis saved to: {analysis_path}")

# Generate RSS feed for podcast distribution
def generate_rss_feed(episodes: List[Dict], output_path: Path):
    """Generate RSS feed for podcast distribution"""
    
    from xml.etree.ElementTree import Element, SubElement, tostring
    from xml.dom import minidom
    
    # Create RSS structure
    rss = Element('rss', version='2.0')
    rss.set('xmlns:itunes', 'http://www.itunes.com/dtds/podcast-1.0.dtd')
    
    channel = SubElement(rss, 'channel')
    
    # Channel information
    SubElement(channel, 'title').text = 'UBMI-IFC Research Frontiers'
    SubElement(channel, 'description').text = 'Exploring cutting-edge research and its connections to institute work'
    SubElement(channel, 'language').text = 'en-us'
    SubElement(channel, 'category').text = 'Science'
    SubElement(channel, 'pubDate').text = datetime.now().strftime('%a, %d %b %Y %H:%M:%S %z')
    
    # Add episodes
    for episode in episodes:
        if episode['status'] == 'completed':
            item = SubElement(channel, 'item')
            
            metadata = episode.get('metadata', {})
            
            SubElement(item, 'title').text = metadata.get('title', f"Episode {episode['episode_number']}")
            SubElement(item, 'description').text = metadata.get('description', 'Research podcast episode')
            SubElement(item, 'pubDate').text = datetime.fromisoformat(metadata.get('generation_date', datetime.now().isoformat())).strftime('%a, %d %b %Y %H:%M:%S %z')
            
            # Add audio enclosure if available
            audio_file = episode.get('files_generated', {}).get('audio_mp3')
            if audio_file and Path(audio_file).exists():
                file_size = Path(audio_file).stat().st_size
                enclosure = SubElement(item, 'enclosure')
                enclosure.set('url', f"file://{audio_file}")  # In production, use actual URL
                enclosure.set('length', str(file_size))
                enclosure.set('type', 'audio/mpeg')
    
    # Pretty print XML
    rough_string = tostring(rss, 'unicode')
    reparsed = minidom.parseString(rough_string)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(reparsed.toprettyxml(indent="  "))

# Generate RSS feed
rss_path = podcast_output_dir / "podcast_feed.xml"
generate_rss_feed(generated_episodes, rss_path)

print(f"\n📡 RSS feed generated: {rss_path}")
print(f"\n🎉 PODCAST GENERATION PIPELINE COMPLETE!")
print("=" * 60)
print(f"📁 All outputs saved to: {podcast_output_dir}")
print(f"📊 Analysis available in: podcast_analysis.json")
print(f"📡 RSS feed ready: podcast_feed.xml")
print(f"\n🚀 Your podcast is ready for distribution!")